# 2. Tarsの確率分布の記述方法

ここではまず，Tarsにおける確率モデルの実装方法について説明します．

In [1]:
from __future__ import print_function
import argparse
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

torch.manual_seed(1)

## 2.1 シンプルな確率分布の設定

ガウス分布を作るためには，``NormalModel``をインポートして，平均（loc）と分散（scale）を定義します．

In [2]:
from Tars.distributions import Normal

x_dim = 50
p1 = Normal(var=["x"], loc=0, scale=1, dim=x_dim)

なお``var``には，変数の名前を設定します．ここでは`"x"`を設定しています．

また，dimでは次元数を指定します．ここではdimが50となっていますから，50次元のサンプルを生成する形になります．

上記で定義したp1の情報は次のようにみることができます．

In [3]:
print(p1.distribution_name) 
print(p1.prob_text) 

Normal
p(x)


distribution_nameでは，確率分布の名前を確認できます．

prob_textでは，確率分布の形をテキストで出力できます．ここでテキストに書かれている確率変数は，上記のvarで指定したものです.


次に，定義した分布からサンプリングしてみましょう． サンプリングは，sampleによって実行します．

In [4]:
samples = p1.sample()
print(samples)
print(samples["x"])

{'x': tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798,
         -1.6091, -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,
          0.2284,  0.4676, -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,
          0.2444, -0.6629,  0.8073,  1.1017, -0.1759, -2.2456, -1.4465,
          0.0612, -0.6177, -0.7981, -0.1316,  1.8793, -0.0721,  0.0663,
         -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,  0.2152, -0.5242,
         -0.1860, -0.6446,  1.5392, -0.8696, -3.3312, -0.7479,  1.1173,
          0.2981]])}
tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798,
         -1.6091, -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,
          0.2284,  0.4676, -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,
          0.2444, -0.6629,  0.8073,  1.1017, -0.1759, -2.2456, -1.4465,
          0.0612, -0.6177, -0.7981, -0.1316,  1.8793, -0.0721,  0.0663,
         -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,  0.2152, -0.5242,
         -0.1860, -0.6446,  1.5392, -

出力はdict形式になっています．

サンプリング結果を確認したい変数について指定することで，中身を確認できます（ただし，この例では変数は"x"のみです）．

なお，サンプリング結果は，PyTorchのtensor形式になっています．

続いて，尤度の計算方法を説明します．
例えば，次のサンプルがあったとしましょう．

In [5]:
_sample1 = torch.Tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085,
         -0.2155,  2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196,
         -0.5484, -0.3472,  0.4730, -0.4286,  0.5514, -1.5474,  0.7575,
         -0.4068, -0.1277,  0.2804,  1.7460,  1.8550, -0.7064,  2.5571,
          0.7705, -1.0739, -0.2015, -0.5603, -0.6240, -0.9773, -0.1637,
         -0.3582, -0.0594, -2.4919,  0.2423,  0.2883, -0.1095,  0.3126,
         -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880, -1.1435,
         -0.6512]])

このサンプルにおける，p1の対数尤度を計算します．

これは，log_likelihoodによって簡単に計算できます．

In [6]:
log_like = p1.log_likelihood({"x":_sample1})
print(log_like)

tensor([-72.5003])


なお，log_likelihoodの引数はdictで与えることに注意してください． これは，どの確率変数かを指定するためです（この例では，xしかありませんが）

TODO: パラメータを変数とする場合の説明を書く

In [7]:
x_dim = 50
_p1 = Normal(cond_var=["mu"], var=["x"], loc="mu", scale=1, dim=x_dim)

In [8]:
print(_p1.distribution_name) 
print(_p1.prob_text) 

Normal
p(x|mu)


In [9]:
_p1.sample({"mu":0})

{'x': tensor(2.2820), 'mu': 0}

In [10]:
_p2 = Normal(var=["mu"], loc=0, scale=1, dim=x_dim)

In [11]:
_p3 = _p1 * _p2
print(_p3.prob_text) 

p(x,mu)


In [12]:
_p3.sample()

{'mu': tensor([[ 1.6734,  0.0103,  0.9837,  0.8793, -1.4504, -1.1802,  0.4100,
           0.4085,  0.3956, -0.9823,  1.3264,  0.8547, -0.2805,  0.7000,
          -1.4567,  1.6089,  0.1716, -0.1600, -0.5047, -1.4746, -0.3416,
          -0.3003,  1.3075, -1.1628,  0.2911,  1.9907, -0.9247, -0.9301,
           1.4301,  0.4208, -0.3538,  0.7639, -0.9276,  1.1120,  0.1573,
           1.2540,  1.3275, -0.4954,  1.5496,  0.3476,  0.0930,  0.6147,
          -0.6447, -0.2870,  3.3212, -0.4021, -0.7123, -0.6200, -0.2281,
          -0.7893]]),
 'x': tensor([[ 0.6363, -1.0566,  0.7752,  0.6638, -1.1799, -0.6205,  0.0916,
           1.9202, -1.1252, -0.0627,  0.7780,  0.5075, -1.0280, -0.2234,
          -0.8833,  1.4996,  0.9291, -0.5668, -0.6324, -1.1942, -0.3041,
          -0.9381,  0.4928, -1.8523,  1.0617,  0.9168, -1.1262, -1.4904,
           2.1119, -0.0962,  1.4364,  1.3516, -0.6771,  0.3189,  0.3995,
           1.5423,  1.2180, -0.1828,  3.0534,  0.8514, -0.4755,  1.4523,
          -0.9303,

## 2.2 深層ニューラルネットワークと組み合わせた確率分布の設定

次に， 確率分布のパラメータを深層ニューラルネットワークで定義します．

例えば，ガウス分布の平均$\mu$と分散$\sigma^2$は， パラメータ$\theta$を持つ深層ニューラルネットワークによって，$\mu=f(x;\theta)$および$\sigma^2=g(x;\theta)$と定義できます．

したがって，ガウス分布は${\cal N}(\mu=f(x;\theta),\sigma^2=g(x;\theta))$となります．

Tarsでは，次のようなクラスを記述することで，これを実現できます．

In [13]:
a_dim = 20

class P2(Normal):
    def __init__(self):
        super(P2, self).__init__(cond_var=["x"], var=["a"])

        self.fc1 = nn.Linear(x_dim, 10)
        self.fc21 = nn.Linear(10, a_dim)
        self.fc22 = nn.Linear(10, a_dim)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return {"loc":self.fc21(h1), "scale":F.softplus(self.fc22(h1))} # mean and variance
    
p2 = P2()

まず， ガウス分布クラスを継承することで，ガウス分布のパラメータを深層ニューラルネットワークで定義することを明示します．

次に，コンストラクタで，利用するニューラルネットワークを記述します．これは，通常のPyTorchと同じです．

唯一異なる点は，superの引数にvarとcond_varの名前を指定している点です．

varは先程見たように，出力する変数の名前を指定します．一方，cond_varではニューラルネットワークの入力変数の名前を指定します．これは，ここで定義する分布において，条件付けられる変数とみなすことができます．

forwardについても，通常のPyTorchと同じです．ただし，注意点が2つあります．

* 引数の名前と数は，cond_varで設定したものと同じにしてください． 例えば，cond_var=["x", "y"]とした場合は，forward(self, x, y)としてください．ただし，引数の順番は変えても構いません．
* 戻り値は，それぞれの確率分布のパラメータになります．上記の例ではガウス分布なので，平均と分散を指定しています．

そして最後に，定義した確率分布クラスのインスタンスを作成します．

次に，先程の例と同様，確率分布の情報を見てみましょう.

In [14]:
print(p2.distribution_name) 
print(p2.prob_text) 

Normal
p(a|x)


prob_textが，先ほどと異なり，xで条件付けた形になっています．これらの表記は，superの引数で設定したとおりになっています．

次に，先程の例のように，サンプリングしてみましょう．

注意しなければならないのは，先ほどと異なり，条件づけた変数xがあるということです．

先程おいた_samplesをxとしてサンプリングしましょう．

In [15]:
samples = p2.sample({"x":_sample1})
print(samples)
print(samples["a"])
print(samples["x"])

{'a': tensor([[ 0.6543,  0.4414,  0.4326,  0.6563,  1.6146,  0.7543, -0.4256,
         -0.5965, -0.5136,  0.3094, -0.2979,  0.2488,  0.0521, -0.5897,
         -0.8235, -0.6019, -0.4913, -0.8328, -0.5683, -0.8692]]), 'x': tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085,
         -0.2155,  2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196,
         -0.5484, -0.3472,  0.4730, -0.4286,  0.5514, -1.5474,  0.7575,
         -0.4068, -0.1277,  0.2804,  1.7460,  1.8550, -0.7064,  2.5571,
          0.7705, -1.0739, -0.2015, -0.5603, -0.6240, -0.9773, -0.1637,
         -0.3582, -0.0594, -2.4919,  0.2423,  0.2883, -0.1095,  0.3126,
         -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880, -1.1435,
         -0.6512]])}
tensor([[ 0.6543,  0.4414,  0.4326,  0.6563,  1.6146,  0.7543, -0.4256,
         -0.5965, -0.5136,  0.3094, -0.2979,  0.2488,  0.0521, -0.5897,
         -0.8235, -0.6019, -0.4913, -0.8328, -0.5683, -0.8692]])
tensor([[-0.3030, -1.7618,  0.6348, -0.8044, 

出力には，aとxの２つのサンプルがあります．

aが今回計算したサンプルで，xについては，引数として与えたサンプルがそのまま入っています．

次に，尤度計算をします．

尤度計算では， 全ての変数のデータを与える必要があります．

aについては，上記でサンプルした値を入れて計算しましょう．

In [16]:
_sample2 = samples["a"]
log_like = p2.log_likelihood({"x":_sample1, "a":_sample2})
print(log_like)

tensor([-19.4727])


なお，これはもっと簡単に書くことができます．

上記のサンプリングで全ての変数とその値がdict形式で出力されたので，それをそのままlog_likelihoodの引数とすればいいのです．

In [17]:
log_like = p2.log_likelihood(samples)
print(log_like)

tensor([-19.4727])


このように記述できる利点は， **変数の数が増えても同じ書き方で尤度計算を実行できる**ことです．

サンプリング->尤度計算という処理は，深層生成モデルでは数多く登場します． このように記述できることで，どのような確率分布の形であっても容易に尤度を計算できるようになるのです．

## 2.3 確率分布の積の設定

最初に書いたとおり，Tarsの特徴の1つは，確率分布の積を簡単に記述できることです．

ここで，これまで設定した確率分布を再度確認しましょう．

In [18]:
print(p1.prob_text) 
print(p2.prob_text) 

p(x)
p(a|x)


数式的には，これらを掛け合わせることで同時分布を定義できます． 

$p(x,a) = p(a|x)p(x)$

では，Tarsではこれをどのように記述するのでしょうか．

実は，それぞれの確率分布のインスタンスを**文字通り掛け合わせるだけ**でいいのです．

In [19]:
p3 = p1 * p2

p3ではどのような確率分布が定義されているか確認しましょう．

In [20]:
print(p3.prob_text) 

p(a,x)


確かに，同時分布が定義されていることがわかります．

このインスタンスp3からも，これまでと同様にサンプリングや尤度計算ができます．

In [21]:
samples = p3.sample()
log_like = p3.log_likelihood(samples)
print(log_like)

tensor([-82.8476])


この例をみてわかるように，サンプリングや尤度計算において，もはやp3を構成する各分布の形を気にする必要はありません．

このような記述方法は，Pythonにおける確率モデリングライブラリでは**Tarsが初めて採用しました**．

これは，確率分布が増えても同じです． 例えば，次の確率分布を定義しましょう．

In [22]:
class P4(Normal):
    def __init__(self):
        super(P4, self).__init__(cond_var=["a", "x"], var=["y"])

        self.fc1 = nn.Linear(x_dim, 10)
        self.fc2 = nn.Linear(a_dim, 10)
        self.fc21 = nn.Linear(10+10, 20)
        self.fc22 = nn.Linear(10+10, 20)

    def forward(self, a, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(a))
        h12 = torch.cat([h1, h2], 1)
        return {"loc":self.fc21(h12), "scale":F.softplus(self.fc22(h12))}
    
p4 = P4()

print(p4.prob_text) 

p(y|a,x)


p4は，aとxで条件付けられた確率分布です．他に$p(a|x)$，$p(x)$をつかって，同時分布は以下のように書けます．

$p(y,a,z)=p(y|a,x)p(a|x)p(x)$

同時分布が3つの確率分布の積になっていますが，これも上記と同様に書けます．

In [23]:
p5 = p4 * p2 * p1
print(p5.prob_text)
print(p5.prob_factorized_text)

p(y,a,x)
p(y|a,x)p(a|x)p(x)


なお，prob_factorized_textは，p5がどのような分布から構成されているかを表示します． サンプリングや尤度計算も全く同じようにできます．

In [24]:
samples = p5.sample()
log_like = p5.log_likelihood(samples)
print(log_like)

tensor([-106.6829])


このように，分布や深層ニューラルネットワークの形を気にせずに，同じ記述方法でサンプリングや尤度計算ができます．

次に，この記述方法を使って，深層生成モデルの1つであるVAEを実装してみましょう．